# Feature + Target Pipeline

Transform hourly ETH/BTC OHLCV data into a ML-ready feature matrix with a directional label.

In [1]:
import sys
from pathlib import Path


def locate_project_root() -> Path:
    current = Path.cwd().resolve()
    for candidate in (current, *current.parents):
        if (candidate / "src").exists():
            return candidate
    raise RuntimeError("Folder 'src' tidak ditemukan dari jalur kerja sekarang.")


PROJECT_ROOT = locate_project_root()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

import numpy as np
import pandas as pd

from src.features import (
    build_technical_features,
    export_feature_dataset,
    load_ethbtc_1h,
    make_forward_return,
    make_forward_return_sign,
)


## Load OHLCV data

In [2]:
raw_df = load_ethbtc_1h()
raw_df.tail()

,open,high,low,close,volume,EMA,ATR,MACD,Signal
time,,,,,,,,,
2025-11-26 03:00:00+00:00,2955.42,2963.74,2920.26,2925.54,183510.531,2932.701884,36.173846,14.610382,14.136613
2025-11-26 04:00:00+00:00,2925.54,2937.86,2918.56,2928.90,117109.053,2932.664054,34.968571,12.690728,13.847436
2025-11-26 05:00:00+00:00,2928.90,2951.70,2928.08,2942.74,102921.296,2932.764312,34.157959,12.146148,13.507179
2025-11-26 06:00:00+00:00,2942.74,2953.81,2930.98,2944.64,115455.344,2932.882478,33.348819,11.732632,13.152269
2025-11-26 07:00:00+00:00,2944.65,2947.42,2937.81,2943.06,17787.429,2932.983747,31.653189,11.148907,12.751597


## Engineer technical features

Notebook kini menambahkan blok return multi-horizon, momentum, volatilitas, perubahan volume,
serta **Average Directional Index (ADX)** lengkap dengan Plus/Minus Directional Indicator untuk
memastikan sinyal tren yang lebih informatif sebelum dataset disimpan.


In [3]:
features = build_technical_features(raw_df)
features.tail()

,ret_1h,ret_4h,ret_12h,ret_24h,momentum_6h,momentum_12h,momentum_24h,momentum_48h,volatility_6h,volatility_24h,volatility_72h,volume_change_6h,volume_change_24h,plus_di_14,minus_di_14,adx_14
time,,,,,,,,,,,,,,,,
2025-11-26 03:00:00+00:00,-0.010158,-0.011115,0.010777,0.005226,-0.001403,0.010836,0.005240,0.024521,0.008299,0.007026,0.007439,0.944702,0.257029,19.428946,21.418810,13.456880
2025-11-26 04:00:00+00:00,0.001148,-0.008222,0.000423,-0.002442,-0.013423,0.000424,-0.002439,0.037252,0.005250,0.006787,0.007439,-0.469944,-0.204798,18.662996,20.921663,12.903240
2025-11-26 05:00:00+00:00,0.004714,-0.008823,0.002685,0.005278,-0.005240,0.002688,0.005292,0.026729,0.005886,0.006826,0.007454,-0.381794,0.098426,20.635303,19.888293,12.113251
2025-11-26 06:00:00+00:00,0.000645,-0.003651,0.007414,0.008030,-0.002858,0.007441,0.008062,0.037645,0.005895,0.006808,0.007419,0.118817,0.193331,20.078196,18.915781,11.460949
2025-11-26 07:00:00+00:00,-0.000537,0.005971,0.007459,0.014201,-0.008677,0.007487,0.014303,0.040403,0.005186,0.006645,0.007380,-0.871890,-0.863572,19.642782,18.505575,10.855239


## Create 4h forward return sign target

In [4]:
HORIZON_HOURS = 4
target = make_forward_return_sign(raw_df, horizon=HORIZON_HOURS)
asset_returns = make_forward_return(
    raw_df, horizon=HORIZON_HOURS, return_type="simple"
).rename(f"asset_return_{HORIZON_HOURS}h")

dataset = features.join([target, asset_returns]).dropna()

default_cutoff = pd.Timestamp("2023-01-01 00:00:00+00:00")
if isinstance(dataset.index, pd.DatetimeIndex) and dataset.index.tz is not None:
    cutoff = default_cutoff.tz_convert(dataset.index.tz)
elif isinstance(dataset.index, pd.DatetimeIndex):
    cutoff = default_cutoff.tz_localize(None)
else:
    cutoff = default_cutoff.tz_localize(None)

if (dataset.index > cutoff).sum() == 0:
    fallback_idx = max(int(len(dataset) * 0.8), 1)
    cutoff = dataset.index[min(fallback_idx, len(dataset) - 1)]

split_labels = np.where(dataset.index <= cutoff, "train", "test")
dataset = dataset.assign(split=split_labels)

cutoff_utc = cutoff.tz_convert("UTC") if getattr(cutoff, "tzinfo", None) else cutoff
dataset_metadata = pd.DataFrame(
    [
        ("horizon_hours", HORIZON_HOURS),
        ("target_column", target.name),
        ("asset_return_column", asset_returns.name),
        ("split_cutoff_utc", cutoff_utc),
        ("train_rows", int((dataset["split"] == "train").sum())),
        ("test_rows", int((dataset["split"] == "test").sum())),
    ],
    columns=["key", "value"],
).set_index("key")
dataset.tail()


,ret_1h,ret_4h,ret_12h,ret_24h,momentum_6h,momentum_12h,momentum_24h,momentum_48h,volatility_6h,volatility_24h,volatility_72h,volume_change_6h,volume_change_24h,plus_di_14,minus_di_14,adx_14,target_sign_return_4h,asset_return_4h,split
time,,,,,,,,,,,,,,,,,,,
2025-11-25 23:00:00+00:00,-0.003546,0.012603,0.023501,0.002047,0.007970,0.023779,0.002049,0.056239,0.006360,0.006982,0.007426,-0.328323,0.519099,22.406714,17.896502,14.475683,0,-0.011054,test
2025-11-26 00:00:00+00:00,-0.001746,0.008621,0.009212,0.003766,0.010329,0.009255,0.003773,0.059709,0.006026,0.006953,0.007433,-0.369282,-0.341880,21.388341,17.963161,14.063425,0,-0.008188,test
2025-11-26 01:00:00+00:00,0.005316,0.013282,0.021363,0.017273,0.016305,0.021593,0.017423,0.058493,0.006055,0.006793,0.007317,-0.548351,0.159451,22.151168,16.556202,14.091360,0,-0.008785,test
2025-11-26 02:00:00+00:00,-0.004527,-0.004504,0.026052,0.008720,0.009454,0.026394,0.008758,0.040916,0.006750,0.006837,0.007330,-0.579539,-0.583300,21.253686,15.885407,14.117301,0,-0.003644,test
2025-11-26 03:00:00+00:00,-0.010158,-0.011115,0.010777,0.005226,-0.001403,0.010836,0.005240,0.024521,0.008299,0.007026,0.007439,0.944702,0.257029,19.428946,21.418810,13.456880,1,0.005989,test


## Persist dataset for ML pipelines

In [5]:
output_dir = PROJECT_ROOT / "data/processed"
output_dir.mkdir(parents=True, exist_ok=True)
parquet_path = output_dir / "ethbtc_1h_features_targets.parquet"
saved_path = export_feature_dataset(dataset, parquet_path)
saved_path


WindowsPath('C:/Users/jefri/backtest/data/processed/ethbtc_1h_features_targets.parquet')

In [6]:
from pathlib import Path
import importlib.util
import pandas as pd
from datetime import datetime, timezone
try:
    PROJECT_ROOT
except NameError:  # pragma: no cover - notebook convenience
    PROJECT_ROOT = Path.cwd()



def export_tables_to_excel(tables, path: Path) -> Path:
    def strip_timezone_from_value(value):
        if value is pd.NaT:
            return value
        if isinstance(value, pd.Timestamp):
            if value.tz is not None:
                return value.tz_convert("UTC").tz_localize(None)
            return value
        if isinstance(value, datetime):
            if value.tzinfo is not None:
                return value.astimezone(timezone.utc).replace(tzinfo=None)
            return value
        return value

    def strip_timezone_from_axis(axis):
        if isinstance(axis, pd.MultiIndex):
            new_levels = [strip_timezone_from_axis(level) for level in axis.levels]
            return axis.set_levels(new_levels)
        if isinstance(axis, pd.DatetimeIndex) and axis.tz is not None:
            return axis.tz_convert("UTC").tz_localize(None)
        if getattr(axis, "dtype", None) == object:
            return pd.Index([strip_timezone_from_value(val) for val in axis], name=axis.name)
        return axis

    def make_excel_safe(frame: pd.DataFrame) -> pd.DataFrame:
        frame = frame.copy()
        frame.index = strip_timezone_from_axis(frame.index)
        frame.columns = strip_timezone_from_axis(frame.columns)
        for column in frame.columns:
            series = frame[column]
            if isinstance(series.dtype, pd.DatetimeTZDtype):
                frame[column] = series.dt.tz_convert("UTC").dt.tz_localize(None)
            elif series.dtype == object:
                frame[column] = series.map(strip_timezone_from_value)
        return frame

    serialisable = []
    for sheet_name, table in tables.items():
        if table is None:
            continue
        if isinstance(table, pd.Series):
            frame = table.to_frame()
        elif isinstance(table, pd.DataFrame):
            frame = table.copy()
        elif isinstance(table, dict):
            frame = pd.DataFrame([table])
        else:
            frame = pd.DataFrame(table)
        frame = make_excel_safe(frame)
        serialisable.append((sheet_name, frame))

    if not serialisable:
        raise ValueError("Tidak ada tabel yang bisa diekspor.")

    path.parent.mkdir(parents=True, exist_ok=True)

    def pick_engine() -> str:
        for candidate in ("openpyxl", "xlsxwriter"):
            if importlib.util.find_spec(candidate):
                return candidate
        raise ModuleNotFoundError(
            "Untuk ekspor Excel diperlukan paket 'openpyxl' atau 'xlsxwriter'."
        )

    def normalise_sheet_name(name: str, existing) -> str:
        safe = (name or "Sheet").strip() or "Sheet"
        safe = safe[:31]
        counter = 1
        candidate = safe
        while candidate in existing:
            suffix = f"_{counter}"
            trimmed = safe[: 31 - len(suffix)] or "Sheet"
            candidate = f"{trimmed}{suffix}"
            counter += 1
        existing.add(candidate)
        return candidate

    engine = pick_engine()
    used_names = set()
    with pd.ExcelWriter(path, engine=engine) as writer:
        for sheet_name, frame in serialisable:
            name = normalise_sheet_name(str(sheet_name), used_names)
            frame.to_excel(writer, sheet_name=name, index=True)

    print(
        f"Berhasil mengekspor {len(serialisable)} sheet ke {path} (engine: {engine})"
    )
    return path
export_dir = PROJECT_ROOT / "outputs" / "result-test"
export_path = export_dir / "features_target_pipeline.xlsx"

export_tables_to_excel(
    {
        "features": features,
        "target": target,
        "dataset": dataset,
        "metadata": dataset_metadata,
    },
    export_path,
)


Berhasil mengekspor 4 sheet ke C:\Users\jefri\backtest\outputs\result-test\features_target_pipeline.xlsx (engine: openpyxl)


WindowsPath('C:/Users/jefri/backtest/outputs/result-test/features_target_pipeline.xlsx')